# CPE 490 Final Project
### By Elijah Moore  

## 1. Acquiring the Data

In [1]:
import yfinance as yf
import datetime as dt

# Stock Tickers
apple = "AAPL"
tesla = "TSLA"
microsoft = "MSFT"
nvidia = "NVDA"
ford = "F"
google = "GOOGL"


# Acquires two years of Historical Data
start_date = dt.datetime.today()- dt.timedelta(days=730) 
end_date = dt.datetime.today()

apple_hist = yf.download(apple, start_date, end_date)
tesla_hist = yf.download(tesla, start_date, end_date)
microsoft_hist = yf.download(microsoft, start_date, end_date)
nvidia_hist = yf.download(nvidia, start_date, end_date)
ford_hist = yf.download(ford, start_date, end_date)
google_hist = yf.download(google, start_date, end_date)


# apple_hist.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-04-08,171.779999,171.779999,169.199997,170.089996,168.168213,76575500
2022-04-11,168.710007,169.029999,165.500000,165.750000,163.877274,72246700
2022-04-12,168.020004,169.869995,166.639999,167.660004,165.765686,79265200
2022-04-13,167.389999,171.039993,166.770004,170.399994,168.474701,70618900
2022-04-14,170.619995,171.270004,165.039993,165.289993,163.422424,75329400


## Model for running 